In [28]:
import pandas as pd

In [29]:
df = pd.read_csv('s3://fintech540-ml/external/layoffs_data.csv')

In [30]:
df.head()

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off
0,IRL,SF Bay Area,Consumer,1.00,2023-06-23,https://www.theinformation.com/articles/social...,197.0,Series C,2023-06-25 23:04:16,United States,NaN,Unknown
1,Retool,SF Bay Area,Other,0.09,2023-06-22,Internal memo,141.0,Series C,2023-06-22 14:23:40,United States,NaN,Unknown
2,Anaplan,SF Bay Area,Other,NaN,2023-06-21,https://nypost.com/2023/06/23/software-giant-a...,300.0,Acquired,2023-06-24 03:48:00,United States,300.0,Unknown
3,Uber,SF Bay Area,Transportation,NaN,2023-06-21,https://www.nasdaq.com/articles/uber-to-lay-of...,25200.0,Post-IPO,2023-06-22 00:19:03,United States,200.0,Unknown
4,Tackle.io,Boise,Infrastructure,NaN,2023-06-21,https://tackle.io/blog/tackle-company-update/,148.0,Series C,2023-06-23 15:38:59,United States,75.0,Unknown


In [34]:
df['Date'] = pd.to_datetime(df['Date'])

df = (df.groupby(['Industry', 'Date'])
        .agg({'Funds_Raised': ['mean', 'sum'], 
              'Laid_Off_Count': ['mean', 'sum']})
        .reset_index())

df.columns = ['industry', 'date', 'funds_raised_mean', 'funds_raised_sum', 
              'laid_off_count_mean', 'laid_off_count_total']

df = df.fillna(0)

In [35]:
df.head()

,industry,date,funds_raised_mean,funds_raised_sum,laid_off_count_mean,laid_off_count_total
0,Aerospace,2020-03-27,3000.0,3000.0,451.0,451.0
1,Aerospace,2020-04-05,100.0,100.0,40.0,40.0
2,Aerospace,2020-04-30,75.0,75.0,0.0,0.0
3,Aerospace,2020-06-03,1.0,1.0,70.0,70.0
4,Aerospace,2022-09-21,1.0,1.0,100.0,100.0


In [36]:
df['industry'].unique()

array(['Aerospace', 'Construction', 'Consumer', 'Crypto', 'Data',
       'Education', 'Energy', 'Finance', 'Fitness', 'Food', 'HR',
       'Hardware', 'Healthcare', 'Infrastructure', 'Legal', 'Logistics',
       'Manufacturing', 'Marketing', 'Media', 'Other', 'Product',
       'Real Estate', 'Recruiting', 'Retail', 'Sales', 'Security',
       'Support', 'Transportation', 'Travel'], dtype=object)

In [38]:
df['sector'] = None

# Classify each industry into one of the six sectors
df.loc[df['industry'].isin(['Aerospace', 'Energy', 'Transportation']), 'sector'] = 'ENERGY & TRANSPORTATION'
df.loc[df['industry'].isin(['Finance', 'Crypto', 'Legal']), 'sector'] = 'FINANCE'
df.loc[df['industry'] == 'Healthcare', 'sector'] = 'LIFE SCIENCES'
df.loc[df['industry'].isin(['Manufacturing', 'Construction', 'Hardware']), 'sector'] = 'MANUFACTURING'
df.loc[df['industry'].isin(['Data', 'Security']), 'sector'] = 'TECHNOLOGY'
df.loc[df['industry'].isin(['Retail', 'Sales', 'Marketing', 'HR', 'Recruiting', 'Support', 'Logistics', 'Food', 'Fitness', 'Travel', 'Education', 'Media', 'Consumer', 'Infrastructure', 'Real Estate', 'Product', 'Other']), 'sector'] = 'TRADE & SERVICES'

# Drop records that do not fit into one of the six sectors
df = df.dropna(subset=['sector'])

In [43]:
df = df[['date', 'sector', 'funds_raised_mean', 'funds_raised_sum', 'laid_off_count_mean', 'laid_off_count_total']]

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2197 entries, 0 to 2196
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  2197 non-null   datetime64[ns]
 1   sector                2197 non-null   object        
 2   funds_raised_mean     2197 non-null   float64       
 3   funds_raised_sum      2197 non-null   float64       
 4   laid_off_count_mean   2197 non-null   float64       
 5   laid_off_count_total  2197 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 103.1+ KB


In [45]:
df

,date,sector,funds_raised_mean,funds_raised_sum,laid_off_count_mean,laid_off_count_total
0,2020-03-27,ENERGY & TRANSPORTATION,3000.0,3000.0,451.0,451.0
1,2020-04-05,ENERGY & TRANSPORTATION,100.0,100.0,40.0,40.0
2,2020-04-30,ENERGY & TRANSPORTATION,75.0,75.0,0.0,0.0
3,2020-06-03,ENERGY & TRANSPORTATION,1.0,1.0,70.0,70.0
4,2022-09-21,ENERGY & TRANSPORTATION,1.0,1.0,100.0,100.0
...,...,...,...,...,...,...
2192,2023-03-01,TRADE & SERVICES,839.0,839.0,100.0,100.0
2193,2023-03-03,TRADE & SERVICES,6400.0,6400.0,30.0,30.0
2194,2023-03-21,TRADE & SERVICES,3300.0,3300.0,0.0,0.0
2195,2023-04-13,TRADE & SERVICES,20.0,20.0,0.0,0.0


In [47]:
df.to_csv('SECTOR_LAYOFFS_DAILY.csv', index=False)